In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
import IPython.display as ipd

In [5]:
train_directory = "/Users/andavaro/Desktop/Andrès/UBA/TrabajoDeGradoCEIA/VENV_Python/Tesis_IA/train/"

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory, labels='inferred', label_mode='int', image_size=(256, 256), seed=123, 
    validation_split=0.2, subset='validation')

Found 16 files belonging to 4 classes.
Using 3 files for validation.


2023-08-23 18:11:33.679518: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-08-23 18:11:33.679552: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-08-23 18:11:33.679558: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-08-23 18:11:33.679602: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-23 18:11:33.679627: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
class_names = train_ds.class_names
print(class_names)

['Copia de VT', 'Copia de VT 2', 'VT', 'VTobspy']


# Modelos

In [15]:
num_classes = len(class_names)
img_height = 256
img_width = 256

## Modelo 1

In [10]:
model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [11]:
learning_rate = 0.125

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ['accuracy']
model.compile(optimizer, loss_fn, metrics)

In [12]:
# Set the epocks
epochs = 10
print('\nFitting:')

# Train the model.
history = model.fit(train_ds, epochs=epochs)


Fitting:
Epoch 1/10


2023-08-23 18:18:25.888509: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 3s 3s/step - loss: 1.4745 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 34ms/step - loss: 4.4370 - accuracy: 0.6667
Epoch 3/10
1/1 [==============================] - 0s 32ms/step - loss: 3.5963 - accuracy: 0.3333
Epoch 4/10
1/1 [==============================] - 0s 31ms/step - loss: 1.8610 - accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 31ms/step - loss: 1.1983 - accuracy: 0.6667
Epoch 6/10
1/1 [==============================] - 0s 32ms/step - loss: 2.2476 - accuracy: 0.6667
Epoch 7/10
1/1 [==============================] - 0s 32ms/step - loss: 1.0362 - accuracy: 0.3333
Epoch 8/10
1/1 [==============================] - 0s 32ms/step - loss: 0.7157 - accuracy: 0.6667
Epoch 9/10
1/1 [==============================] - 0s 36ms/step - loss: 0.9852 - accuracy: 0.3333
Epoch 10/10
1/1 [==============================] - 0s 31ms/step - loss: 1.0511 - accuracy: 0.6667


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 64, 64, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        1

### Test

In [14]:
correct = 0
batch_size = 0
for batch_num, (X, Y) in enumerate(train_ds):
    batch_size = len(Y)
    pred = model.predict(X)
    for i in range(batch_size):
        predicted = np.argmax(pred[i], axis=-1)
        actual = Y[i]
        #print(f'predicted {predicted}, actual {actual}')
        if predicted == actual:
            correct += 1
    break

print(f'Number correct: {correct} out of {batch_size}')
print(f'Accuracy {correct / batch_size}')

1/1 [==============================] - 0s 149ms/step
Number correct: 2 out of 3
Accuracy 0.6666666666666666


2023-08-23 18:19:33.825637: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


## Modelo 2

In [24]:
'''import tensorflow_decision_forests as tfdf
model = tfdf.keras.RandomForestModel(
    core.Task.CLASSIFICATION
)'''
model2 = tf.keras.models.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [25]:
learning_rate = 0.125

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ['accuracy']
model.compile(optimizer, loss_fn, metrics)

In [26]:
# Set the epocks
epochs = 15
print('\nFitting:')

# Train the model.
history = model.fit(train_ds, epochs=epochs)


Fitting:
Epoch 1/15


2023-08-23 18:34:29.552314: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 264ms/step - loss: 0.9592 - accuracy: 0.6667
Epoch 2/15
1/1 [==============================] - 0s 36ms/step - loss: 0.9780 - accuracy: 0.6667
Epoch 3/15
1/1 [==============================] - 0s 32ms/step - loss: 0.8247 - accuracy: 0.6667
Epoch 4/15
1/1 [==============================] - 0s 32ms/step - loss: 0.8678 - accuracy: 0.6667
Epoch 5/15
1/1 [==============================] - 0s 53ms/step - loss: 0.7089 - accuracy: 0.6667
Epoch 6/15
1/1 [==============================] - 0s 33ms/step - loss: 0.6776 - accuracy: 0.6667
Epoch 7/15
1/1 [==============================] - 0s 32ms/step - loss: 0.6657 - accuracy: 0.6667
Epoch 8/15
1/1 [==============================] - 0s 31ms/step - loss: 0.6838 - accuracy: 0.6667
Epoch 9/15
1/1 [==============================] - 0s 38ms/step - loss: 0.7183 - accuracy: 0.3333
Epoch 10/15
1/1 [==============================] - 0s 32ms/step - loss: 0.8003 - accuracy: 0.6667
Epoch 11/15
1/1 [======================

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 64, 64, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        1

### Test

In [29]:
correct = 0
batch_size = 0
for batch_num, (X, Y) in enumerate(train_ds):
    batch_size = len(Y)
    pred = model.predict(X)
    for i in range(batch_size):
        predicted = np.argmax(pred[i], axis=-1)
        actual = Y[i]
        #print(f'predicted {predicted}, actual {actual}')
        if predicted == actual:
            correct += 1
    break

print(f'Number correct: {correct} out of {batch_size}')
print(f'Accuracy {correct / batch_size}')

1/1 [==============================] - 0s 51ms/step
Number correct: 2 out of 3
Accuracy 0.6666666666666666


2023-08-23 18:34:45.293035: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
